<a href="https://colab.research.google.com/github/BharathkumarK19/Bert_pre-trained/blob/main/FINE_Tunning_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-Tunning the Bert with custom Data

In [ ]:
#Fine-Tunning with IMDB Data set

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd

In [3]:
od.download('https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews')

Skipping, found downloaded files in "./imdb-dataset-of-50k-movie-reviews" (use force=True to force download)


In [4]:
df=pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [6]:
df=df.head(3000)

In [7]:
df.shape

(3000, 2)

In [8]:
df['sentiment'].value_counts()

,count
sentiment,
positive,1508
negative,1492


set up of Bert

In [13]:
tf.__version__

'2.17.1'

In [ ]:
!pip install tensorflow==2.10.0
!pip install ktrain
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import ktrain

In [10]:
from ktrain import text
import tensorflow as tf
import keras

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_data,test_data=train_test_split(df,test_size=0.5,random_state=4519)

In [13]:
train_data.shape,test_data.shape

((1500, 2), (1500, 2))

In [14]:
#splitting for the training and testing data
(x_train,y_train),(x_test,y_test),prepro=text.texts_from_df(
    train_df=train_data,
    text_column='review',
    label_columns='sentiment',
    val_df=test_data,
    maxlen=400,
    preprocess_mode = 'bert')


['negative', 'positive']
      negative  positive
120        0.0       1.0
1313       1.0       0.0
1404       1.0       0.0
785        1.0       0.0
695        1.0       0.0
['negative', 'positive']
      negative  positive
2859       1.0       0.0
90         0.0       1.0
297        0.0       1.0
674        1.0       0.0
2777       1.0       0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [15]:
model=text.text_classifier(name='bert',
                           train_data=(x_train,y_train),
                           preproc=prepro)

Is Multi-Label? False
maxlen is 400


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [16]:
learner=ktrain.get_learner(model=model,
                           train_data=(x_train,y_train),
                           val_data=(x_test,y_test),
                           batch_size=5)

In [17]:
learner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
300/300 [==============================] - 6428s 21s/step - loss: 0.4557 - accuracy: 0.7747 - val_loss: 0.2514 - val_accuracy: 0.9013


Our training accuracy is less because we are trained with less data we don't have enough Graphic Card

In [18]:
predictor=ktrain.get_predictor(learner.model, prepro)

In [19]:
predictor.save('/content/drive/My Drive/bert')

In [22]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again',
        'waste of time by watching this movie']

In [23]:
predictor.predict(data)

['negative', 'negative', 'positive', 'negative']